
[![Open In Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/waleedgeo/gee_py_ml/blob/main/gee_py_ml.ipynb)



## Machine Learning Classification with Google Earth Engine Python API

### recap Some basic python vs javascript syntax for earth engine


|Property|JavaScript|Python|
|---|---|---|
|Function definition|function myFun(arg) {  return arg;  <br>}  <br>  <br>var myFun = function(arg) {  return arg;  <br>};|def my_fun(arg):  return arg|
|Anonymous function mapping|var foo = col.map(function(arg) {  return arg;  <br>});|foo = col.map(lambda arg: arg)|
|Variable definition|var myVar = 'var';|my_var = 'var'|
|Logical operators|var match = such.and(that);  <br>var match = such.or(that);  <br>var match = such.not(that);|match = such.And(that)match = such.Or(that)match = such.Not(that)|
|Multi-line method chain|var foo = my.really()              .reallyLong()              .methodChain();|foo = (my.really()       .reallyLong()       .methodChain())|
|Dictionary keys|var dic = {'key': value};  <br>var dic = {key: value};|dic = {'key': value}|
|Dictionary object access|var value = dic.key;  <br>var value = dic['key'];|value = dic['key']|
|Function argument definition|// Positional arguments.  <br>var foo = fun(argX, argY, argZ);  <br>// Keyword arguments object.  <br>var foo = fun({y: argY});|# Positional arguments.foo = fun(arg_x, arg_y, arg_z)  <br># Keyword arguments dictionary.foo = fun(**{'y': arg_y})  <br># Keyword arguments.foo = fun(x=arg_x, z=arg_z)|
| Boolean | var t = true; var f = false; | t = Truef = False|
|Null values|var na = null;|na = None|
|Comment|//|#|

# rewrite above table in markdown with refined syntax


Source (API Difference): [GEE Guide](https://developers.google.com/earth-engine/guides/python_install#syntax-differences-between-the-javascript-and-python-apis)

### Full Classification in Earth Engine (using ESRI LULC as label layer)

In [1]:
!pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=5aa9b895f19c1011afeac1d0c1de407e307c3cf55dafdc2f8abac205dbb96e23
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236756 sha256=fcb6d1fdb68dc3f1c37ac782e14811d9351000989668f625a83bd45dd216ffc3
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [ ]:
import ee
import geemap
import geemap.colormaps as cm
import geopandas as gpd
import eemont
import os

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [3]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

loc = '/content/drive/MyDrive/Colab Notebooks/gee_py_ml/'
# create a path if it doesn't exist
if not os.path.exists(loc):
    os.makedirs(loc)
os.chdir(loc)
print('Working Directory: ',os.getcwd())

Mounted at /content/drive
Working Directory:  /content/drive/MyDrive/Colab Notebooks/gee_py_ml


In [4]:
point = ee.Geometry.Point(3.3896932773281856, 6.516974260768799) #Lagos, Nigeria
aoi = point.buffer(5000).bounds() # 5km buffer around the point


Map = geemap.Map()
Map.centerObject(aoi, 12)
Map.addLayer(aoi, {}, 'AOI')

In [5]:
#def mask_s2_clouds(image):
#  """Masks clouds in a Sentinel-2 image using the QA band.
#
#  Args:
#      image (ee.Image): A Sentinel-2 image.
#
#  Returns:
#      ee.Image: A cloud-masked Sentinel-2 image.
#  """
#  qa = image.select('QA60')
#
#  # Bits 10 and 11 are clouds and cirrus, respectively.
#  cloud_bit_mask = 1 << 10
#  cirrus_bit_mask = 1 << 11
#
#  # Both flags should be set to zero, indicating clear conditions.
#  mask = (
#      qa.bitwiseAnd(cloud_bit_mask)
#      .eq(0)
#      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
#  )
#
#  return image.updateMask(mask).divide(10000)

s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2023-01-01', '2023-01-15')
    # Pre-filter to get less cloudy granules.
    #.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 60))
    .filterBounds(aoi)
    #.map(mask_s2_clouds)
    .maskClouds()
    .scaleAndOffset()
    .spectralIndices(['MNDVI', 'NDBI', 'MNDWI'])
    .mean()
    .clip(aoi)
)

# ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')

vis_fcc = {
    'min': 0.2,
    'max': 0.5,
    'bands': ['B8', 'B4', 'B3'],
}

bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12', 'MNDVI', 'NDBI', 'MNDWI']
image = s2.select(bands)


Map.add_layer(image, vis_fcc, 'FCC', False)
# visualize spectral indices
Map.addLayer(image.select('MNDVI'), {'min': -0.2, 'max': 0.3, 'palette': cm.palettes.ndvi}, 'MNDVI', False)
Map.addLayer(image.select('NDBI'), {'min': -0.2, 'max': 0.1, 'palette': cm.get_palette('coolwarm')}, 'NDBI', False)
Map.addLayer(image.select('MNDWI'), {'min': -0.3, 'max': 0.1, 'palette': cm.get_palette('RdBu')}, 'MNDWI', False)



### ESRI LULC Link: https://gee-community-catalog.org/projects/S2TSLULC/

![ESRI LULC](https://user-images.githubusercontent.com/6677629/187349140-d571fe8e-e979-48cf-a1a1-2f3c06b957a0.gif)


In [6]:

# create a reference image and sample from it
esri_lulc = (ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS")
                .filterDate('2022-01-01', '2022-12-31')
                .mosaic()
                .clip(aoi)
                .remap([1,2,4,5,7,8,9,10,11],[1,2,3,4,5,6,7,8,9])
)

esri_vis = {
  "names": [
    "Water",
    "Trees",
    "Flooded Vegetation",
    "Crops",
    "Built Area",
    "Bare Ground",
    "Snow/Ice",
    "Clouds",
    "Rangeland"
  ],
  "colors": [
    "#1A5BAB",
    "#358221",
    "#87D19E",
    "#FFDB5C",
    "#ED022A",
    "#EDE9E4",
    "#F2FAFF",
    "#C8C8C8",
    "#C6AD8D"
  ]}
Map.addLayer(esri_lulc, {'min':1, 'max':9, 'palette':esri_vis['colors']}, '2022 LULC 10m', False)

field = esri_lulc.remap([1, 5],[2, 1]).rename('lc')

Map.addLayer(field, {'min':1, 'max':2, 'palette':['red', 'blue']}, 'NEW ESRI Field', False)
#Map


In [8]:
# projections of image and new reference image
print('Image projection:', image.projection().getInfo())
print('Reference projection:', field.projection().getInfo())

Image projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}
Reference projection: {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [10]:
# generate random samples (stratifed by class)
sample_size = 1000
seed = 42
scale = 10
samples = field.stratifiedSample(
    numPoints=sample_size,
    classBand='lc',
    region=aoi,
    scale=scale,
    projection=image.projection(),
    seed=seed,
    geometries=True
)

Map.addLayer(samples, {}, 'samples', False)
print(samples.size().getInfo())
print(samples.first().getInfo())


2000
{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [3.3707035248374746, 6.478604913305783]}, 'id': '0', 'properties': {'lc': 1}}


In [11]:
# sample the image
samples = image.sampleRegions(
    collection=samples,
    properties=['lc'],
    scale=scale,
    projection=image.projection(),
    geometries=True
)
samples.first().getInfo()

{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Point',
  'coordinates': [3.3707035248374746, 6.478604913305783]},
 'id': '0_0',
 'properties': {'B11': 0.28350000000000003,
  'B12': 0.2403,
  'B2': 0.201,
  'B3': 0.22660000000000002,
  'B4': 0.23570000000000002,
  'B5': 0.26195,
  'B6': 0.28790000000000004,
  'B7': 0.30425,
  'B8': 0.2982,
  'MNDVI': 0.10819010776457585,
  'MNDWI': -0.11102368908281263,
  'NDBI': -0.025656033147810106,
  'lc': 1}}

In [ ]:
# split the samples into training and testing
split = 0.7
samples = samples.randomColumn()

training = samples.filter(ee.Filter.lt('random', split))
testing = samples.filter(ee.Filter.gte('random', split))


# train a classifier
classifier = ee.Classifier.smileRandomForest(numberOfTrees=100).train(
    features=training,
    classProperty='lc',
    inputProperties=bands
)

# classify the image
classified = image.select(bands).classify(classifier)
Map.addLayer(classified, {'min':1, 'max':2, 'palette':['red', 'blue']}, 'classified')
Map

In [13]:
# Train Accuracy

train_accuracy = classifier.confusionMatrix()
print('Train Confusion Matrix:', train_accuracy.getInfo())

print('Train Overall Accuracy:', train_accuracy.accuracy().getInfo())

print('Producer\'s Accuracy:', train_accuracy.producersAccuracy().getInfo())
print('Consumer\'s Accuracy:', train_accuracy.consumersAccuracy().getInfo())



# Test Accuracy
test_accuracy = testing.classify(classifier).errorMatrix('lc', 'classification')

print('Test Confusion Matrix:', test_accuracy.getInfo())
print('Test Overall Accuracy:', test_accuracy.accuracy().getInfo())
print('Producer\'s Accuracy:', test_accuracy.producersAccuracy().getInfo())
print('Consumer\'s Accuracy:', test_accuracy.consumersAccuracy().getInfo())


Train Confusion Matrix: [[0, 0, 0], [0, 718, 0], [0, 0, 695]]
Train Overall Accuracy: 1
Producer's Accuracy: [[0], [1], [1]]
Consumer's Accuracy: [[0, 1, 1]]
Test Confusion Matrix: [[0, 0, 0], [0, 280, 0], [0, 3, 302]]
Test Overall Accuracy: 0.9948717948717949
Producer's Accuracy: [[0], [1], [0.9901639344262295]]
Consumer's Accuracy: [[0, 0.9893992932862191, 1]]


### Manual Training Samples

In [14]:
Map

Map(bottom=505565.0, center=[6.5169822257490875, 3.389765029252635], controls=(WidgetControl(options=['positio…

In [ ]:
vegetation = Map.user_rois
vegetation

In [ ]:
urban = Map.user_rois
urban

In [15]:
water = Map.user_rois
water

In [ ]:
barren = Map.user_rois
barren

In [ ]:
combined_samples = water.merge(urban).merge(vegetation).merge(barren)
combined_samples_gdf = geemap.ee_to_gdf(combined_samples)


In [ ]:
combined_samples_gdf


,geometry,color,label,lc
0,"POLYGON ((3.41638 6.52597, 3.41895 6.52597, 3....",#1a36c1,water,1
1,"POLYGON ((3.41844 6.48060, 3.42187 6.48060, 3....",#1a36c1,water,1
2,"POLYGON ((3.42307 6.49493, 3.42616 6.49493, 3....",#1a36c1,water,1
3,"POLYGON ((3.43148 6.52546, 3.43285 6.52546, 3....",#1a36c1,water,1
4,"POLYGON ((3.42994 6.54268, 3.43131 6.54268, 3....",#1a36c1,water,1
5,"POLYGON ((3.43114 6.55445, 3.43303 6.55445, 3....",#1a36c1,water,1
6,"POLYGON ((3.39304 6.47617, 3.39853 6.47617, 3....",#1a36c1,water,1
7,"POLYGON ((3.38737 6.47327, 3.38875 6.47327, 3....",#1a36c1,water,1
8,"POLYGON ((3.41243 6.51232, 3.41552 6.51232, 3....",#1a36c1,water,1
9,"POLYGON ((3.40969 6.49987, 3.41329 6.49987, 3....",#1a36c1,water,1


In [ ]:
combined_samples_gdf.to_file('combined_samples.shp')

In [16]:
samples_url = 'https://github.com/waleedgeo/gee_py_ml/blob/main/samples/combined_samples.geojson'
samples_out_name = 'samples.geojson'
geemap.download_file(samples_url, samples_out_name)
load_samples = gpd.read_file(samples_out_name)
#print(load_samples.head())
combined_samples = geemap.gdf_to_ee(load_samples)
combined_samples

Downloading...
From: https://raw.githubusercontent.com/waleedgeo/gee_py_ml/main/samples/combined_samples.geojson
To: /content/drive/MyDrive/Colab Notebooks/gee_py_ml/samples.geojson
11.2kB [00:00, 32.8MB/s]                   


In [17]:
# convert samples vectors to images
water = image.select('B2').gt(0).clip(combined_samples.filter(ee.Filter.eq('label', 'water'))).multiply(1).rename('lc').int()
urban = image.select('B2').gt(0).clip(combined_samples.filter(ee.Filter.eq('label', 'urban'))).multiply(2).rename('lc').int()
vegetation = image.select('B2').gt(0).clip(combined_samples.filter(ee.Filter.eq('label', 'vegetation'))).multiply(3).rename('lc').int()
barren = image.select('B2').gt(0).clip(combined_samples.filter(ee.Filter.eq('label', 'barren'))).multiply(4).rename('lc').int()

field = ee.ImageCollection([water, urban, vegetation, barren]).mosaic()

In [18]:
# generate random samples (stratifed by class)
sample_size = 200
seed = 42
scale = 10
samples = field.stratifiedSample(
    numPoints=sample_size,
    classBand='lc',
    region=aoi,
    scale=scale,
    projection=image.projection(),
    seed=seed,
    geometries=True
)

#Map.addLayer(samples, {}, 'samples', False)
print(samples.size().getInfo())
print(samples.first().getInfo())
#Map

746
{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [3.414720973759331, 6.513190051744385]}, 'id': '0', 'properties': {'lc': 1}}


In [19]:
# sample the image
samples = image.sampleRegions(
    collection=samples,
    properties=['lc'],
    scale=scale,
    projection=image.projection(),
    geometries=True
)
samples.first().getInfo()

{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Point',
  'coordinates': [3.414720973759331, 6.513190051744385]},
 'id': '0_0',
 'properties': {'B11': 0.15705000000000002,
  'B12': 0.14335,
  'B2': 0.18030000000000002,
  'B3': 0.19970000000000002,
  'B4': 0.19525,
  'B5': 0.1995,
  'B6': 0.18195,
  'B7': 0.18315,
  'B8': 0.17855000000000001,
  'MNDVI': 0.10650216429018629,
  'MNDWI': 0.126243188353341,
  'NDBI': -0.06430742985190455,
  'lc': 1}}

In [20]:
# split the samples into training and testing
split = 0.7
samples = samples.randomColumn()

training = samples.filter(ee.Filter.lt('random', split))
testing = samples.filter(ee.Filter.gte('random', split))


# train a classifier
classifier = ee.Classifier.smileRandomForest(numberOfTrees=100).train(
    features=training,
    classProperty='lc',
    inputProperties=bands
)

# classify the image
classified = image.select(bands).classify(classifier)
Map.addLayer(classified, {'min':1, 'max':4, 'palette':['blue', 'red', 'green', 'orange']}, 'classified')
Map

Map(bottom=505565.0, center=[6.516931297326792, 3.389968872070313], controls=(WidgetControl(options=['position…

In [21]:
# Train Accuracy

train_accuracy = classifier.confusionMatrix()
print('Train Confusion Matrix:', train_accuracy.getInfo())

print('Train Overall Accuracy:', train_accuracy.accuracy().getInfo())

print('Producer\'s Accuracy:', train_accuracy.producersAccuracy().getInfo())
print('Consumer\'s Accuracy:', train_accuracy.consumersAccuracy().getInfo())



# Test Accuracy
test_accuracy = testing.classify(classifier).errorMatrix('lc', 'classification')

print('Test Confusion Matrix:', test_accuracy.getInfo())
print('Test Overall Accuracy:', test_accuracy.accuracy().getInfo())
print('Producer\'s Accuracy:', test_accuracy.producersAccuracy().getInfo())
print('Consumer\'s Accuracy:', test_accuracy.consumersAccuracy().getInfo())


Train Confusion Matrix: [[0, 0, 0, 0, 0], [0, 151, 0, 0, 0], [0, 0, 146, 0, 0], [0, 0, 1, 140, 0], [0, 0, 0, 0, 105]]
Train Overall Accuracy: 0.998158379373849
Producer's Accuracy: [[0], [1], [1], [0.9929078014184397], [1]]
Consumer's Accuracy: [[0, 1, 0.9931972789115646, 1, 1]]
Test Confusion Matrix: [[0, 0, 0, 0, 0], [0, 49, 0, 0, 0], [0, 0, 54, 0, 0], [0, 0, 1, 55, 3], [0, 0, 0, 0, 41]]
Test Overall Accuracy: 0.9802955665024631
Producer's Accuracy: [[0], [1], [1], [0.9322033898305084], [1]]
Consumer's Accuracy: [[0, 1, 0.9818181818181818, 1, 0.9318181818181818]]


In [27]:
exp_img = classified.toUint8()
geemap.ee_export_image(exp_img, filename='classified1.tif', scale=20, region=aoi)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Colab Notebooks/gee_py_ml/classified1.tif


In [ ]:
# exporting classified image to drive
geemap.ee_export_image_to_drive(classified.toUint8(), description='classified_lagos', folder='geemap', region=aoi, scale=10)

In [28]:
# check the last task status and progress
task = ee.batch.Task.list()[-1]
task.status()


{'state': 'COMPLETED',
 'description': 'denmark_pop',
 'creation_timestamp_ms': 1701229287852,
 'update_timestamp_ms': 1701229596094,
 'start_timestamp_ms': 1701229295456,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1IdAdOqszGzXM8ltp9UP2AjikbWrXnR0L'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 99.57250213623047,
 'id': 'MY7AIM6DD5NSDZUK6PGYR47M',
 'name': 'projects/earthengine-legacy/operations/MY7AIM6DD5NSDZUK6PGYR47M'}

### Hybrid Workflow for ML Classification

In hybrid workflow we do:

1. Export features (either all individual bands as images, or one stacked image with all bands)
2. Export Labels as CSV
3. Reshape the data (images) using Rasterio
4. Train the model using scikit-learn
5. Predict the model using scikit-learn
6. Export the predicted image using Rasterio

In [ ]:
# Exporting our feature images as single band
# bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12', 'MNDVI', 'NDBI', 'MNDWI']
# image = s2.select(bands)

# exporting each band as a single band image
for band in bands:
    geemap.ee_export_image_to_drive(image.select(band), description=band, folder='geemap', region=aoi, scale=10)
    print(band, ' exported')

# export samples as csv
geemap.ee_to_csv(samples, filename='samples_final.csv')

## Follow my this blog for local ML Classification

[![Medium](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*8-rPe8vbjjbm1lhuhcn3Dw.png)](https://waleedgeo.medium.com/lulc-py-78cb954673d)